<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>**
This contains all the data preprocessing and cleaning</p>

In [208]:
import os
print(os.getcwd())

/content


In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [210]:
df = pd.read_csv("concrete_data.csv", skiprows=1)

In [211]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [212]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [213]:
df.dropna(inplace = True)
df.isnull().sum()

,0
cement,0
blast_furnace_slag,0
fly_ash,0
water,0
superplasticizer,0
coarse_aggregate,0
fine_aggregate,0
age,0
concrete_compressive_strength,0


In [214]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [215]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>**

In [216]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [217]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [218]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [219]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [220]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [221]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [222]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x

    # def elu(x, alpha=1.0):
    #     return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

    def visualize_layers(self):
        for i in range(len(self.layerSize)-1):
            print("LAYER COUNT:  ", i,"    NODES: ",self.layerSize[i])
            print(f"  Weights Shape:   ", self.weights[i].shape)
            print(f"  Biases Shape:    ",self.biases[i].shape )
            print(f"  Weights= : {self.weights[i]}")
            print(f"  Biases= {self.biases[i]}")

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)
        return output

Layer size is structure to be 8,16,8,1 because
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [223]:
layer_sizes = [8, 200, 80, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)
ann.visualize_layers()

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

LAYER COUNT:   0     NODES:  8
  Weights Shape:    (8, 200)
  Biases Shape:     (1, 200)
  Weights= : [[ 0.81954054  0.70015638  0.46976722 ... -0.33448275 -0.80097843
  -1.0596443 ]
 [ 0.30281077  0.63626624 -1.10712988 ... -0.53600297 -0.20287332
  -0.39304874]
 [-0.12460351  0.16205261  0.74306384 ...  0.0868167  -0.101571
   0.0476848 ]
 ...
 [ 0.08465374 -0.149151    1.06567797 ... -0.83388345 -0.2170393
  -0.94487632]
 [ 0.19527727 -0.21693245 -1.28319596 ...  0.43457964  0.35280157
  -1.13692455]
 [ 0.95203855  0.06010958 -0.38539358 ... -1.1458798  -0.81203925
   0.1750741 ]]
  Biases= [[ 0.32588303 -1.70191784 -1.36742919 -0.0330081   0.02178314  0.4586939
   0.04448342  1.19415578  2.98910097 -2.10621997  0.04328728 -1.06867791
   1.8774924  -2.64442192  1.98937165 -0.17062572  0.02096038  0.05690273
  -0.2851215  -1.04615433  0.71205505  0.70624054 -2.0757054  -0.37966367
   0.50987219  1.25069421  0.7052317   0.37441374  0.77720717  0.57802659
   0.16409584 -0.42293924  2.2

**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MSE is utilised as the loss function

In [224]:
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train
class MeanAbsoluteError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean(np.abs(y_pred - y_train))

In [225]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN
# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape)
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss_function=MeanAbsoluteError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 34.88134269471461


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [226]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [227]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue

        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange

        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue

        # activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [228]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x)
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [229]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, gamma, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.gamma =gamma
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):

        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # Update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )
                print("updted v:", updatedVelocity)
                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition

<p style="font-size:18px;">Test out PSO Functionality</p>

In [230]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(100, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(100, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO
swarmSize = 50
alpha = 0.9  # inertia weight
beta = 0.7   # cognitive parameter
gamma = 0.5  # social parameter
delta = 0.5
jumpSize = 0.5
informantCount = 2
annLayers=layer_sizes
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 100

# initialize the PSO optimiser
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    gamma = gamma,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimisation
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,

    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function= MeanAbsoluteError(),
    max_iterations = max_iterations
)
print("Optimised ANN Parameters:", best_parameters)

Streaming output truncated to the last 5000 lines.
updted v: [-2.26746077  0.56479977 -1.43138789 ...  1.90031565 -0.37231729
 -0.55406424]
updted v: [-1.05972695 -0.73556891  0.78050959 ...  1.12315419 -1.55242275
  0.01861646]
updted v: [-0.3496338  -0.56145352 -1.7991557  ... -0.20589392 -0.69759552
 -0.82887075]
updted v: [ 1.64649033  0.35716886 -0.84662102 ...  1.00324904  0.71891049
 -0.75982551]
updted v: [-1.51259296  0.4060872  -0.24848131 ... -1.95006381 -2.03387768
 -0.93287527]
updted v: [-0.69525659 -0.28827571 -0.69704096 ...  0.38779215 -0.23555234
 -0.81593029]
updted v: [-3.97150016  0.28163147 -4.30601123 ... -1.92597685 -3.56480273
 -2.55355757]
updted v: [-0.70098056 -0.07225865  0.29577415 ...  0.60526572 -0.56425431
 -0.14341143]
updted v: [-1.17671599  0.19793184  0.49248997 ... -0.21177927 -0.16280054
 -0.68390005]
updted v: [-0.5692256  -0.52781334 -1.23869534 ... -0.23330643 -0.48230932
 -0.56311935]
updted v: [ 0.40845167 -0.33737213  0.68073974 ...  0.37253

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimised by the PSO

In [231]:
# create a dummy particle and assign the optimised parameters
optimised_particle = Particle(vectorSize)
optimised_particle.particlePosition = best_parameters

# convert the optimised particle position back to ANN weights and biases
optimised_ann = particleToAnn(
    optimised_particle,
    annLayers,
    activation_functions
)

# run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimised_particle,  # pass the optimised particle for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function = MeanAbsoluteError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 0.3669872205111207


<p style="font-size:18px;">Training the ANN</p>

In [232]:
y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)
loss_function =MeanAbsoluteError()
training_loss = loss_function.evaluate(y_train_pred, y_train)

print(f"Training Loss: {training_loss}")

Training Loss: 0.3669872205111207


<p style="font-size:18px;">Testing the model on the test set</p>

In [235]:
percentage_error = (test_loss / y_test.mean()) * 100
print(f"Percentage Error: {percentage_error:.2f}%")

Percentage Error: 4.12%


In [234]:
import matplotlib.pyplot as plt

epochs = 100  # Set this to the number of epochs you want
max_iterations = 100
training_losses = []
test_losses = []

for epoch in range(epochs):
    # Forward pass to get predictions
    y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)

    # Compute the training loss
    loss_function = MeanAbsoluteError()
    training_loss = loss_function.evaluate(y_train_pred, y_train)

    # Record the training loss
    training_losses.append(training_loss)

    # Make predictions on the test set
    y_test_pred = optimised_ann.forwardPropagation(X_test_scaled)
    y_test=y_test.reshape(-1, 1)  # Reshape to (n_samples, 1)
    test_loss = loss_function.evaluate(y_test_pred, y_test)

    # Record the test loss
    test_losses.append(test_loss)

    # Print the losses
    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss}")
    print(f"Test Loss: {test_loss}")

# Plot the loss over epochs
plt.plot(range(epochs), training_losses, label='Training Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss over Epochs')
plt.legend()
plt.show()


AttributeError: 'Series' object has no attribute 'reshape'